In [1]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17705325161689039449
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4990763008
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4358933421110885137
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660, pci bus id: 0000:26:00.0, compute capability: 7.5"
]


In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [5]:
# 사전 학습된 모델 불러오기
input_tensor = Input(shape=(150,150,3))
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

vgg_model.summary()

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

In [6]:
for layers in vgg_model.layers:
            layers.trainable=False
        
flatten_layer = Flatten()
dense_layer_1 = Dense(256, activation='relu')
prediction_layer  = Dense(149, activation='softmax')
        
model = Sequential([
    vgg_model,
    flatten_layer,
    dense_layer_1,
    prediction_layer
])
model.summary()

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 149)               38293     
Total params: 16,850,389
Trainable params: 2,135,701
Non-trainable params: 14,714,688
____________________________________________________________

In [7]:
# 컴파일 옵션
model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
# loss = 'sparse_categorical_crossentropy',

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [8]:
# 테스트 데이터 (고양이/강아지)
train_dir ='D:/빅데이터기반지능형서비스개발/06ai서비스개발pjt/data_split/train'
test_dir = 'D:/빅데이터기반지능형서비스개발/06ai서비스개발pjt/data_split/val'

# 폴더에 따라 자동 분류
train_image_generator = ImageDataGenerator(rescale=1./255)
val_image_generator = ImageDataGenerator(rescale=1./255)
### augmentation : ImageDataGenerator( rescale = 1.0/255,shear_range=0.2,zoom_range=0.2)

train_data_gen = train_image_generator.flow_from_directory(batch_size=32,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           color_mode="rgb",
                                                           target_size=(150, 150),
                                                           class_mode='categorical')

val_data_gen = val_image_generator.flow_from_directory(batch_size=32,
                                                         directory=test_dir,
                                                       color_mode="rgb",
                                                         target_size=(150, 150),
                                                         class_mode='categorical')

Found 95475 images belonging to 149 classes.
Found 24049 images belonging to 149 classes.


In [9]:
# 모델 학습
history = model.fit(train_data_gen,
                    epochs=50,
                    validation_data=val_data_gen,
                    validation_steps=100)


model.save("logs/VGG16_tr_1000_try_1_epo_50.h5")


  ...
    to  
  ['...']
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
  ...
    to  
  ['...']
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Train for 2984 steps, validate for 100 steps
Epoch 1/50
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/

C:\Users\kk135\Anaconda3\envs\tf\lib\site-packages\PIL\Image.py:946: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


2003/2984 [===================>..........] - ETA: 4:42 - loss: 3.5995 - accuracy: 0.1698

C:\Users\kk135\Anaconda3\envs\tf\lib\site-packages\PIL\TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


2983/2984 [============================>.] - ETA: 0s - loss: 3.4063 - accuracy: 0.2007Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_10662 in device /job:localhost/replica:0/task:0/device:GPU:0
2984/2984 [==============================] - 1084s 363ms/step - loss: 3.4064 - accuracy: 0.2007 - val_loss: 2.9277 - val_accuracy: 0.2872
Epoch 2/50
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0
2983/2984 [============================>.] - ETA: 0s - loss: 2.7519 - accuracy: 0.3095Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:

KeyboardInterrupt: 

In [ ]:
# 최종 결과 리포트
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

from matplotlib import pyplot as plt

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='testing acc')
plt.title('Training and testing accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='testing loss')
plt.title('Training and testing loss')
plt.legend()

plt.show()

In [50]:
# 모델 불러오기
from keras.models import load_model

new_model = load_model("newVGG16.h5")

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'vgg16/flatten/Reshape' defined at (most recent call last):
    File "C:\Users\kk135\Anaconda3\envs\bigdata\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\kk135\Anaconda3\envs\bigdata\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\kk135\Anaconda3\envs\bigdata\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\kk135\Anaconda3\envs\bigdata\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\kk135\Anaconda3\envs\bigdata\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\kk135\Anaconda3\envs\bigdata\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\kk135\Anaconda3\en

In [3]:
# vgg16 
from keras.applications.vgg16 import VGG16
conv_base = VGG16(
 weights='imagenet',
 include_top=False,
 input_shape=(150,150,3)
)
conv_base.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [3]:
import tensorflow as tf

tf.__version__


'2.8.0'

In [6]:
tf.keras.applications.vgg16.VGG16(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000,
    classifier_activation='softmax'
)
